<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Training neural network model
We will train our image classifier to detect two classes
'free' and 'blocked', we will use them to avoid collisions, and before that we simply used the popular deep learning library in the simple gesture recognition chapter of Pytorch's first experience. *PyTorch* uses Res- Net18 is classified,

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

# Upload and extract data sets
The dataset we collected last time and packaged into a .zip, then extracted (decompressed) this dataset by calling the shell (command line) command. If you are training directly on jetson nano, you can skip this step.

In [ ]:
!unzip -q dataset.zip

After unzipping, you should see a folder called dataset appear in the file browser.

# Create a dataset instance

Now, we use the ImageFolder dataset class in the torchvision.datasets library. There is an additional torchvision.transforms library for converting data to prepare for the training model.

In [ ]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

# Split the data sets into training sets and test sets
Next, we split the data set into a training set and a test set. The test set will be used to verify the accuracy of our trained models.

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

# Create a data loader to load data in bulk

We will create two DataLoader instances that provide utilities for shuffling data, generate batch images, and load samples in parallel with multiple tasks.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

# Defining neural networks

In [ ]:
model = models.alexnet(pretrained=True)

The alexnet model was originally trained for datasets with 1000 class labels, but our dataset has only two class labels!

We will replace the best layer with the latest, untrained layer with only two outputs.

In [ ]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Transfer the model from the CPU to the GPU via 'CUDA' 

In [ ]:
device = torch.device('cuda')
model = model.to(device)

# Training neural network
Using the code below, we will begin to train our neural network and save the best performing model after each generation.

> One generation is to run all the data again

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

When the model is trained, we can see the model named best_model.th generated in the catalog. We will use this model for active obstacle avoidance in the next routine.